1. Напишите программу, которая находит самое длинное слово.

In [3]:
import pyspark

In [4]:
from pyspark import SparkContext

sc = SparkContext()
text_file = sc.textFile("/home/daniel/wiki.txt")


def is_valid_word(word):
    if word.__contains__("http"):
        return False
    if any(char.isdigit() for char in word):
        return False
    return True

longest_word = text_file.flatMap(lambda line: line.split(" ")) \
                        .filter(is_valid_word) \
                        .reduce(lambda x, y: x if len(x) > len(y) else y)

print("Самое длинное слово:", longest_word)
sc.stop()


23/06/05 18:36:39 WARN Utils: Your hostname, Yoga14 resolves to a loopback address: 127.0.1.1; using 192.168.0.113 instead (on interface wlp2s0)
23/06/05 18:36:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/05 18:36:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Самое длинное слово: Галогенарены-Нафталин-Антрацен-Фенантрен-Бензпирен-Коронен-Азулен-Бифенил-Ионол.


2. Напишите программу, которая находит среднюю длину слов.

In [5]:
from pyspark import SparkContext

sc = pyspark.SparkContext()
text_file = sc.textFile("/home/daniel/wiki.txt")

avg_word_length = text_file.flatMap(lambda line: line.split(" "))
    
def validate_word(word):
    if word == "-":
        return 0
    elif word.endswith(".") or word.endswith(",") or word.endswith(":") or word.endswith("?") or word.endswith("!") or word.endswith(";"):
        return len(word) - 1
    else:
        return len(word)
    
lineLengths = avg_word_length.map(lambda s: validate_word(s))
totalLength = lineLengths.reduce(lambda a, b: a + b)
print("Средняя длина слова:", totalLength / avg_word_length.count())
sc.stop()

Средняя длина слова: 6.433347173833233


3. Напишите программу, которая находит самое частоупотребляемое слово, состоящее из латинских букв.

In [18]:
import re

sc = SparkContext()
lines = sc.textFile("/home/daniel/wiki.txt")

words = lines.flatMap(lambda line: re.findall(r'\b(?!ru|http|org)[a-zA-Z]+\b', line))
lowercase_words = words.map(lambda word: word.lower())
word_counts = lowercase_words.countByValue()
most_frequent_word = max(word_counts, key=word_counts.get)

print("Самое частоупотребляемое слово:", most_frequent_word)
sc.stop()

Самое частоупотребляемое слово: wikipedia


4. Все слова, которые более чем в половине случаев начинаются с большой буквы и встречаются больше 10 раз.

In [7]:
from pyspark import SparkConf
conf = SparkConf().setAppName("task 4.4")
sc = SparkContext(conf=conf)
lines = sc.textFile("/home/daniel/wiki.txt")
words = lines.flatMap(lambda line: line.split())
big_words = words.filter(lambda word: word[0].isupper() and word != word.upper())
word_counts = big_words.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
frequent_words = word_counts.filter(lambda x: x[1] > 10)
finally_words = frequent_words.filter(lambda x: sum(1 for word in x[0] if word.isupper()) / len(x[0]) > 0.5)
for word, count in finally_words.collect():
    print("{}: {}".format(word, count))
sc.stop()

НЭПа: 26
СПИДа: 13
РСДРП(б),: 22
ГУЛАГа: 11
UltraSPARC: 20
РСДРП(б): 33
МВт: 46
ВУЗов: 17
ЮКОСа: 40
BeOS: 26
ГГц: 19
OpenBSD: 13
ЗаПЧЕЛ: 20
SNMPv1: 12
СвДП: 33
МПа: 27
HCl: 21
МГц: 70
FreeBSD: 57
СПбГУ: 25
ГэВ: 13
NetBIOS: 12
СПИДом: 11
NSFNet: 19
NaOH: 14
МэВ: 15
TeX: 24
NetBSD: 14
MySQL: 43


5. Напишите программу, которая с помощью статистики определяет устойчивые сокращения вида `пр.`, `др.`, ...

In [59]:
conf = SparkConf().setAppName("task 4.5")
sc = SparkContext(conf=conf)
lines = sc.textFile("/home/daniel/wiki.txt")
words = lines.flatMap(lambda line: line.split())
words = lines.flatMap(lambda line: re.findall(r'[а-яА-Я]+\.', line))
lowercase_words = words.filter(lambda word: not word[0].isupper() and len(word) <= 3 and len(word) > 2)
finally_words = {}
for word in lowercase_words.collect():
    if word in finally_words:
        finally_words[word] += 1
    else:
        finally_words[word] = 1

n = 200
for word in finally_words:
    if finally_words[word] > n:
        print(word)
sc.stop()

см.
др.
ст.
км.
гг.
мм.
кг.
ок.
св.
вв.
кв.
им.
пр.
га.
ул.


6. Напишите программу, которая с помощью статистики определяет устойчивые сокращения вида  `т.п.`, `н.э.`, ...

In [65]:
conf = SparkConf().setAppName("task 4.5")
sc = SparkContext(conf=conf)
lines = sc.textFile("/home/daniel/wiki.txt")
words = lines.flatMap(lambda line: line.split())
words = lines.flatMap(lambda line: re.findall(r'[а-яА-Я]+\.+[а-яА-Я]+\.', line))
lowercase_words = words.filter(lambda word: not word[0].isupper() and len(word) <= 4 and len(word) > 2)
finally_words = {}
for word in lowercase_words.collect():
    if word in finally_words:
        finally_words[word] += 1
    else:
        finally_words[word] = 1


n = 10
for word in finally_words:
    if finally_words[word] > n:
        print(word)
sc.stop()

н.э.
т.д.
т.п.
с.ш.
в.д.
ю.ш.
г.р.
г.г.
т.е.
т.к.
т.н.
л.с.
т.ч.
э.д.
